In [1]:
import openpyxl
from openpyxl.styles import Alignment
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import os
import json
import pandas as pd

In [7]:
def detailinforequest(rrdcNo):
# URL của API
    url = 'https://unipass.customs.go.kr/clip/prlstclsfsrch/retrievePrlstClsfCaseDtl.do'

    # Headers
    headers = {
        'Accept': 'text/html, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'WMONID=N_4A3dINH0I; JSESSIONID=0001O2y8S-yyQQ_lga-knblPV6Lb2QN6662ivRFJ_jK2FPRlLOTKNEsnuhZe_QxBEKq-8U03_kah8EgUgb-_Exa4dmevvhJHEKeNuIq9prRxpJht6ugFASENgz3p_AdMPw4_:eul21',
        'DNT': '1',
        'Origin': 'https://unipass.customs.go.kr',
        'Referer': 'https://unipass.customs.go.kr/clip/index.do',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'isAjax': 'true',
        'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Dữ liệu trong body của yêu cầu
    payload = {
        'cntyCd':'US',
        'baseYy':'2005',
        'reffNoNm':'NY L81580',
        'prlstClsfSrno':'655'
    }

    # Gửi yêu cầu POST với headers và dữ liệu body
    response = requests.post(url, headers=headers, data=payload)
    
    # Kiểm tra phản hồi từ API
    if response.status_code == 200:
        return response.text
    else:
        print(rrdcNo + " Lỗi")



In [8]:
htmlcontent = detailinforequest(1)

In [15]:
soup = BeautifulSoup(htmlcontent, 'html.parser')

In [16]:
soup


<script src="/clip/js/lightbox/js/lightbox.js" type="text/javascript"></script>
<script src="/clip/js/kul/uls_template.js" type="text/javascript"></script>
<script type="text/javascript">
prgm_id = "ULS0203029S";
</script>
<script src="/clip/js/kul/prlstclsfsrch/ULS0203029S.js" type="text/javascript"></script>
<input class="savedtoken" name="ULS0203029S_F1_savedToken" type="hidden" value="EPDQAOOPDL10P0ZF97SZ7A6XJO9U47EA"/>
<input class="savedtoken" name="savedToken" type="hidden" value="ULS0203029S_F1_savedToken"/>
<div class="subtitle clear">
<p class="check floatRight mrgnbttm5">
<span class="btn_abovetable">
<button class="btn_print" type="button"><span>인쇄</span></button>
</span>
</p>
</div>
<div class="pop_layer" id="ULS0203029S_T1_popLayer1" style="position:absolute;top:0;left:0;width:130px;padding:10px;display:none;">
<h3 style="width:109px;min-height:40px;height:auto !important;height:40px;margin:-9px 0 0px -9px;padding:0 20px;line-height:40px;font-size:14px;font-weight:bold;b

In [22]:
print(soup.find("div", class_ = "table_left table_prlstclsf org"))

<div class="table_left table_prlstclsf org">
<table class="org">
<caption>
<strong>품목분류사례-미국 상세결과</strong>
<p>상세결과 항목 : 기준년도, 참조번호, 시행일자, 결정일자, 결정세번, 시행기관, Category, 회신, 물품설명, 이슈장소, 키워드, 관련이미지</p>
</caption>
<colgroup>
<col style="width:20%"/>
<col/>
</colgroup>
<tbody>
<tr>
<th scope="row" width="20%">기준년도</th>
<td class="textLeft">2005</td>
</tr>
<tr>
<th scope="row">참조번호</th>
<td class="textLeft hlzone1">NY L81580</td>
</tr>
<tr>
<th scope="row">시행일자</th>
<td class="textLeft">
							2005-01-13
						</td>
</tr>
<tr>
<th scope="row">결정세번</th>
<td class="textLeft hlzone2"><a class="dtrmLink" data-baseyy="2018" data-cntycd="US" data-dtrmlink="99028403" data-prlsttpcd="02" href="#" style="color:#4271cd !important; font-weight:bold; vertical-align: top !important;" title="9902.84.03">9902.84.03</a>; <a class="dtrmLink" data-baseyy="2024" data-cntycd="US" data-dtrmlink="8401400000" data-prlsttpcd="02" href="#" style="color:#4271cd !important; font-weight:bold; vertical-align: top !import

In [ ]:
 

# Hàm trích xuất các thành phần từ HTML
def extract_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Tìm các thành phần theo nhãn <tr> và trích xuất giá trị
    ref_no = soup.find('th', text='참조번호').find_next_sibling('td').text.strip()
    implementation_date = soup.find('th', text='시행일자').find_next_sibling('td').text.strip()
    implementing_agency = soup.find('th', text='시행기관').find_next_sibling('td').text.strip()

    # Chỉ lấy phần đầu tiên của quyết định (trước dấu xuống dòng \n)
    decision_code = soup.find('th', text='결정세번').find_next_sibling('td').find('input')['value'].strip()

    product_name_html = soup.find('th', text='품명').find_next_sibling('td')
    for elem in product_name_html(True):
        if elem.name == 'a':  # Đổi thẻ <a> thành văn bản
            elem.replace_with(elem.text)
        elif elem.name == 'br':  # Giữ lại thẻ <br>
            continue
    product_name = product_name_html.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()

    product_description_html = soup.find('th', text='물품설명').find_next_sibling('td')
    for elem in product_description_html(True):
        if elem.name == 'a':  # Đổi thẻ <a> thành văn bản
            elem.replace_with(elem.text)
        elif elem.name == 'br':  # Giữ lại thẻ <br>
            continue
    product_description = product_description_html.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()    
        
    decision_reason_html = soup.find('th', text='결정사유').find_next_sibling('td')
    for elem in decision_reason_html(True):
        if elem.name == 'a':  # Đổi thẻ <a> thành văn bản
            elem.replace_with(elem.text)
        elif elem.name == 'br':  # Giữ lại thẻ <br>
            continue
    decision_reason = decision_reason_html.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()    

    image_count = soup.find('th', text='이미지건수').find_next_sibling('td').text.strip()
    # Liệt kê các hình ảnh trong mục "관련 이미지"
    image_elements = soup.find('th', text='관련 이미지').find_next_sibling('td').find_all('img')
    image_links = [img['src'] for img in image_elements]

    # Tạo kết quả dưới dạng dictionary
    result = {
        "참조번호": ref_no,
        "시행일자": implementation_date,
        "시행기관": implementing_agency,
        "결정세번": decision_code,
        "품명": product_name,
        "물품설명": product_description,
        "결정사유": decision_reason,
        "이미지건수": image_count,
        "관련 이미지": image_links
    }
    return result

In [14]:
extract_data_from_html(htmlcontent)

C:\Users\nguye\AppData\Local\Temp\ipykernel_19808\982939226.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ref_no = soup.find('th', text='참조번호').find_next_sibling('td').text.strip()
C:\Users\nguye\AppData\Local\Temp\ipykernel_19808\982939226.py:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  implementation_date = soup.find('th', text='시행일자').find_next_sibling('td').text.strip()
C:\Users\nguye\AppData\Local\Temp\ipykernel_19808\982939226.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  implementing_agency = soup.find('th', text='시행기관').find_next_sibling('td').text.strip()
C:\Users\nguye\AppData\Local\Temp\ipykernel_19808\982939226.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  decision_code = soup.find('th', text='결정세번').find_next_sibling('td').find('i

TypeError: 'NoneType' object is not subscriptable